## 내용


## 컬럼들 

1.airlines.csv (2개 컬럼) :항공사데이터
-IATA_CODE :IATA 공항 코드
세계의 모든 공항이 IATA에 의해 중복되지 않게 코드를 부여
ex) 김포공항은 GMP, 인천국제공항은 ICN과 같은 형식
-AIRLINE:항공사

2.airport.csv (7개 컬럼)  :공항데이터

*IATA_CODE :공항코드
*AIRPORT:공항
*CITY  :도시
*STATE :미국 50개주
*COUNTRY: 전부 미국
*LATITUDE :위도 
*LONGITUDE:경도

3.flight.csv (31개의 컬럼) :비행기 데이터
*YEAR :연
*MONTH : 월
*DAY :일자
*DAY_OF_WEEK :요일 (1~7 로 구성 : 7일이 토요일인지 생각해봐야함)
*AIRLINE :항공사
*FLIGHT_NUMBER :비행기 번호
*TAIL_NUMBER : 항공기 등록기호 
 *항공기형태에 따른 등록기호번호지정:http://blog.naver.com/PostView.nhn?blogId=hjy8169&logNo=130026878725&redirect=Dlog&widgetTypeCall=true
*ORIGIN_AIRPORT : 출국공항
*DESTINATION_AIRPORT :입국공항
*SCHEDULED_DEPARTURE :출발예정시간 (20~600 범위)
*DEPARTURE_TIME :출발시간
*DEPARTURE_DELAY :출발지연
*TAXI_OUT : 출발 공항 게이트에서 출발하고 휠을 끄는 사이에 경과한 시간.
*WHEELS_OFF :항공기 바퀴가 지면을 벗어나는 시점.
*SCHEDULED_TIME : 40~250
*ELAPSED_TIME :경과시간
*AIR_TIME :휠소프와 휠_ON 시간 사이의 시간
*DISTANCE :거리
*WHEELS_ON : 항공기 바퀴가 지면에 닿는 시점.
*TAXI_IN :휠 온과 목적지 공항 게이트 도착 사이에 경과한 시간.
*SCHEDULED_ARRIVAL :도착예정 400~800 
*ARRIVAL_TIME :도착시간
*ARRIVAL_DELAY :도착지연
*DIVERTED : 우회 (아마 회항일듯)
*CANCELLED :결항
--------null 존재 행------------------------
84,91,...null 값처리는 아마 아예 빼야할듯.
------------아래부터 null값 다수------------

CANCELLATION_REASON :결항이유
AIR_SYSTEM_DELAY :
SECURITY_DELAY :보안지연
AIRLINE_DELAY :항공사지연
LATE_AIRCRAFT_DELAY :
WEATHER_DELAY :날짜지연

 ### 시간형 데이터 형식: 다음 시간은 xx:yy - hour:min 형식으로 되어 있다(예: 1536은 오후 3시 36분, 345는 오전 3시 45분, 16은 00시 16분)
 
 ### 데이터 관계 (교호작용 고랴)
*arrivaltime = wheelson + taxi_in
*arrivaldelay = arrivaltime - *scheduled_arrival
*departuretime = wheelsoff - taxi_out
*departuredelay = departuretime - *scheduled_departure
*elapsedtime =airtime + taxiin + taxiout
*airtime = wheelson - wheels_off

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%matplotlib inline
import datetime, warnings, scipy 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")

In [ ]:
airlines = pd.read_csv('../input/airlines.csv')
airports= pd.read_csv('../input/airports.csv')
flights = pd.read_csv('../input/flights.csv', low_memory=False)
print (airlines.shape) # 14행 2열
print (airports.shape) # 332행 7열
print (flights.shape)  #5819079행 31열

In [ ]:
#결항(canclelled)컬럼의 평균과 비행기의 개수의 곱
print (flights.shape[0]*flights.CANCELLED.mean())
print (flights.shape[0] - flights.CANCELLATION_REASON.isnull().sum().sum())

두 숫자가 일치하는데, 이는 취소된 비행기들만 취소 이유를 가지고 있다는 것을 암시한다. 직관적!

In [ ]:
print (flights['ARRIVAL_DELAY'][flights['ARRIVAL_DELAY'] >= 15].count())
print (flights.shape[0] - flights.AIR_SYSTEM_DELAY.isnull().sum().sum())

이 두 수치는 일치하며, 도착 지연이 있는 비행만 15분에서 상세한 지연 분해(예: 항공 시스템, 항공사, 날씨)를 갖도록 한다.

In [ ]:
flights_v1 = pd.merge(flights, airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')
flights_v1.drop('IATA_CODE', axis=1, inplace=True)
flights_v1.rename(columns={'AIRLINE_x': 'AIRLINE_CODE','AIRLINE_y': 'AIRLINE'}, inplace=True)

# Delay plots

In [ ]:
airport_mean_delays = pd.DataFrame(pd.Series(flights['ORIGIN_AIRPORT'].unique()))
airport_mean_delays.set_index(0, drop = True, inplace = True)
abbr_companies = airlines.set_index('IATA_CODE')['AIRLINE'].to_dict()
identify_airport = airports.set_index('IATA_CODE')['CITY'].to_dict()

# function that extract statistical parameters from a grouby objet:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
#___________________________________________________________

for carrier in abbr_companies.keys():
    fg1 = flights[flights['AIRLINE'] == carrier]
    test = fg1['DEPARTURE_DELAY'].groupby(flights['ORIGIN_AIRPORT']).apply(get_stats).unstack()
    airport_mean_delays[carrier] = test.loc[:, 'mean'] 

In [ ]:
sns.set(context="paper")
fig = plt.figure(1, figsize=(12,15))

ax = fig.add_subplot(1,2,1)
subset = airport_mean_delays.iloc[:50,:].rename(columns = abbr_companies)
subset = subset.rename(index = identify_airport)
mask = subset.isnull()
sns.heatmap(subset, linewidths=0.05, cmap="YlGnBu", mask=mask, vmin = 0, vmax = 30)
plt.setp(ax.get_xticklabels(), fontsize=12, rotation = 88) ;
ax.yaxis.label.set_visible(False)

ax = fig.add_subplot(1,2,2)    
subset = airport_mean_delays.iloc[50:100,:].rename(columns = abbr_companies)
subset = subset.rename(index = identify_airport)
fig.text(0.5, 1.02, "Scale of Delays from origin airport", ha='center', fontsize = 20)
mask = subset.isnull()
sns.heatmap(subset, linewidths=0.05, cmap="YlGnBu", mask=mask, vmin = 0, vmax = 30)
plt.setp(ax.get_xticklabels(), fontsize=12, rotation = 88) ;
ax.yaxis.label.set_visible(False)

plt.tight_layout()

2015년 항공편 수가 가장 많은 항공사는 사우스웨스트(1.2M편)와 델타, 아메리칸, 스카이웨스트 등이며, 취소율도 비교적 낮은 것으로 보인다. 아메리칸 이글은 취소율이 가장 높다 - 약 5%. 

또한, 방향 전환 속도는 비행 체적과 긍정적인 상관관계가 있는 것으로 보인다. 전체적으로 방향 전환율은 1% 미만이다

In [ ]:
font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 15}
mpl.rc('font', **font)
import matplotlib.patches as mpatches
#__________________________________________________________________
# extract a subset of columns and redefine the airlines labeling 
fg2 = flights.loc[:, ['AIRLINE', 'DEPARTURE_DELAY']]
fg2['AIRLINE'] = fg2['AIRLINE'].replace(abbr_companies)
fig = plt.figure(1, figsize=(16,16))
gs=GridSpec(1,2)             
ax1=fig.add_subplot(gs[0,0]) 
ax2=fig.add_subplot(gs[0,1]) 

global_stats_d=flights['DEPARTURE_DELAY'].groupby(flights['AIRLINE']).apply(get_stats).unstack()
global_stats_d=global_stats_d.sort_values('count')

global_stats_a=flights['ARRIVAL_DELAY'].groupby(flights['AIRLINE']).apply(get_stats).unstack()
global_stats_a=global_stats_a.sort_values('count')

#----------------------------------------
# Pie chart nº1: mean delay at departure
#----------------------------------------
labels = [s for s in  global_stats_d.index]
sizes  = global_stats_d['mean'].values
sizes  = [max(s,0) for s in sizes]
explode = [0.0 if sizes[i] < 20000 else 0.01 for i in range(len(abbr_companies))]
patches, texts, autotexts = ax1.pie(sizes, explode = explode, labels = labels, shadow=False, startangle=0, 
                                    autopct = lambda p :  '{:.0f}'.format(p * sum(sizes) / 100))
for i in range(len(abbr_companies)): 
    texts[i].set_fontsize(18)
ax1.axis('equal')
ax1.set_title('Mean delay at departure', bbox={'facecolor':'midnightblue', 'pad':10},
              color='w', fontsize=18)

#----------------------------------------
# Pie chart nº2: mean delay at arrival
#----------------------------------------
labels = [s for s in  global_stats_a.index]
sizes  = global_stats_a['mean'].values
sizes  = [max(s,0) for s in sizes]
explode = [0.0 if sizes[i] < 20000 else 0.01 for i in range(len(abbr_companies))]
patches, texts, autotexts = ax2.pie(sizes, explode = explode, labels = labels, shadow=False, startangle=0,
                                     autopct = lambda p :  '{:.0f}'.format(p * sum(sizes) / 100))
for i in range(len(abbr_companies)): 
    texts[i].set_fontsize(18)
ax2.axis('equal')
ax2.set_title('Mean delay at arrival', bbox={'facecolor':'midnightblue', 'pad':10},
              color='w', fontsize=18)

#Flight Volume, Cancellation & Divertion Rate :비행 물량, 취소 및 운항률